In [ ]:
import pandas as pd
import numpy as np

# %% Imports
from pathlib import Path
import sys
root_dir = Path().resolve()
sys.path.append(str(root_dir / 'src'))

from recsys_common import *


In [ ]:
meta=get_metadata()

In [ ]:
sessions=get_sessions(True,.05,True,0.25)
sessions['impressions']=sessions['impressions'].str.split('\\|')
sessions['prices']=sessions['prices'].str.split('\\|')
print(sessions.dtypes)
sessions.head()

# Get Splits

In [ ]:
test = sessions.loc[sessions.is_train==False].reset_index(drop=True)
# train = sessions.loc[(sessions.is_train==True)]
train = sessions.loc[(sessions.is_train==True) & (sessions.is_validation==False)].reset_index(drop=True)
dev = sessions.loc[(sessions.is_train==True) & (sessions.is_validation==True)].reset_index(drop=True)

print('test',test.shape)
print('train',train.shape)
print('dev',dev.shape)

sessions.loc[(sessions.is_validation==True)].reset_index(drop=True)

# Check number of test clickouts without a reference

In [ ]:
nclickouts_nan = test.loc[(test.action_type=='clickout item') & (test.reference.isna()),'reference'].shape[0]
nclickouts = test.loc[(test.action_type=='clickout item'),'reference'].shape[0]
print('test clickouts without reference, total, and frac:', [nclickouts_nan,nclickouts,nclickouts_nan/nclickouts])

# Get Fake Test Set

In [ ]:
dev_test,gt = process_validation(dev, frac_nan=0.488, seed=1234)
dev_test.head()
gt.head()

# Munge for NB

In [ ]:
colnames=dev_test.columns
colnames

Filter for numeric and NaN references and drop columns 'platform','city','device','current_filters','fake_split_train','is_train'

In [ ]:
dev_test_filt=dev_test.loc[(dev_test.reference.isnull()) | (dev_test.reference.str.contains('^[0-9]*$'))].drop(['platform','city','device','current_filters','is_validation','is_train'],axis=1).copy()
print(dev_test_filt.shape)
dev_test_filt

In [ ]:
# collect last 5 actions for clickouts step >=6
dev_test_filt['len_session']=dev_test_filt.groupby('session_id')['step'].transform('count')
indx_stepbig_nan=dev_test_filt.loc[(dev_test_filt.action_type=='clickout item') & (dev_test_filt.reference.isnull()) & (dev_test_filt.len_session>=6)].index.astype(int)
# indx_stepbig_nan=dev_test_filt.loc[(dev_test_filt.action_type=='clickout item') & (dev_test_filt.reference.isnull()) & (dev_test_filt.step>=6)].index.astype(int)
print(len(indx_stepbig_nan)*5)
indx_stepbig_nan_range = ([])
step_num=([])
for i in indx_stepbig_nan:
#     print(np.arange(i-5,i))
    indx_stepbig_nan_range=np.concatenate((indx_stepbig_nan_range,np.arange(i-5,i)))
    step_num = np.concatenate((step_num,np.arange(1,6)))
    
print([len(indx_stepbig_nan_range),len(indx_stepbig_nan_range)==len(step_num)])
len(step_num)

# temp2=dev_test_filt.loc[dev_test_filt.index.isin(indx_stepbig_nan_range.astype(np.int64))].copy()
# temp2


In [ ]:
grouped=dev_test_filt.groupby('session_id')
grouped.filter(lambda x: x[(dev_test_filt.action_type=='clickout item') & (dev_test_filt.reference.isnull())])

In [ ]:
has_clickout_nan = dev_test_filt.loc[(dev_test_filt.action_type=='clickout item') & (dev_test_filt.reference.isnull()),['session_id','step']]
has_clickout_nan
# dev_test_filt.loc[(dev_test_filt.session_id.isin(has_clickout_nan.session_id)) & (dev_test_filt.step<=has_clickout_nan.step)]

dev_test_filt[lambda x: x['session_id'].isin(has_clickout_nan.session_id) & x['step']<=has_clickout_nan.step]

In [ ]:
(train.loc[~train.impressions.isnull(),'impressions'].apply(lambda x: len(x)))